In [7]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
df_rec = pd.read_csv("../datasets/epi_r.csv")
df_rec

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
list_of_food = [
    'almond', 'amaretto', 'anchovy', 'anise', 'apple', 'apricot', 'artichoke', 'arugula', 'asparagus', 'avocado',
    'bacon', 'banana', 'barley', 'basil', 'beef', 'beet', 'bell pepper', 'berry', 'blackberry', 'blue cheese',
    'blueberry', 'bok choy', 'bran', 'bread', 'brie', 'broccoli', 'bulgur', 'burrito', 'butter', 'buttermilk',
    'butternut squash', 'cabbage', 'candy', 'cantaloupe', 'capers', 'carrot', 'cashew', 'cauliflower', 'caviar',
    'celery', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea', 'chile pepper', 'chili', 'chive',
    'chocolate', 'coconut', 'cod', 'coriander', 'corn', 'crab', 'cranberry', 'cream cheese', 'cucumber', 'curry',
    'custard', 'dairy', 'date', 'duck', 'egg', 'eggplant', 'endive', 'fennel', 'feta', 'fig', 'fish', 'garlic',
    'goat cheese', 'gouda', 'grape', 'grapefruit', 'green bean', 'ham', 'hamburger',
    'hazelnut', 'honey', 'hummus', 'ice cream', 'jalapeño', 'kale', 'kiwi', 'lamb', 'lemon', 'lentil', 'lettuce',
    'lima bean', 'lime', 'lobster', 'macaroni and cheese', 'mango', 'maple syrup', 'mayonnaise', 'meatball',
    'melon', 'mint', 'mushroom', 'mussel', 'mustard', 'nutmeg', 'oatmeal', 'olive', 'omelet', 'onion', 'orange',
    'oregano', 'oyster', 'pancake', 'papaya', 'paprika', 'parmesan', 'parsley', 'parsnip', 'pasta', 'peanut',
    'pear', 'pecan', 'pepper', 'persimmon', 'pineapple', 'pistachio', 'pizza', 'plum', 'pomegranate', 'pork',
    'potato', 'poultry', 'prosciutto', 'prune', 'pumpkin', 'quail', 'quinoa', 'radish', 'raisin', 'raspberry',
    'rice', 'ricotta', 'rosemary', 'salmon', 'salsa', 'sausage', 'scallop', 'seafood', 'sesame', 'shallot',
    'shrimp', 'spinach', 'squash', 'steak', 'strawberry', 'sugar snap pea', 'swiss cheese',
    'tangerine', 'tapioca', 'tarragon', 'tea', 'thyme', 'tilapia', 'tofu', 'tomato', 'trout', 'tuna', 'turnip',
    'vanilla', 'veal', 'vegetable', 'walnut', 'wasabi', 'watermelon', 'wild rice', 'yellow squash', 'yogurt',
    'zucchini'
]

In [4]:
nutrients_list = {
    'Vitamin A': 900,
    'Vitamin C': 90,
    'Calcium': 1300,
    'Iron': 18,
    'Vitamin D': 20,
    'Vitamin E': 15,
    'Vitamin K': 120,
    'Thiamin': 1.2,
    'Riboflavin': 1.3,
    'Niacin': 16,
    'Vitamin B6': 1.7,
    'Folate': 400,
    'Vitamin B12': 2.4,
    'Biotin': 30,
    'Pantothenic acid': 5,
    'Phosphorus': 1250,
    'Iodine': 150,
    'Magnesium': 420,
    'Zinc': 11,
    'Selenium': 55,
    'Copper': 0.9,
    'Manganese': 2.3,
    'Chromium': 35,
    'Molybdenum': 45,
    'Chloride': 2300,
    'Potassium': 4700,
    'Choline': 550,
    'Fat': 78,
    'Saturated fat': 20,
    'Cholesterol': 300,
    'Total carbohydrates': 275,
    'Sodium': 2300,
    'Dietary Fiber': 28,
    'Protein': 50,
    'Added sugars': 50
}


In [5]:
def getInfo(ingr):
    r = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', 
                params={'api_key':'Cuqs4rsZCvPhO6lxaQ8gJGpQNFH2jun6sxumNhvx',
                        'query':ingr})
    json = r.json()
    nutr_dict = {}
    for nutr in json['foods'][0]['foodNutrients']:
        name = nutr['nutrientName'].split(',')[0]
        if name in nutrients_list.keys():
            nutr_dict[name] = nutr['value']/nutrients_list[name]*100
    return nutr_dict


In [14]:
with ThreadPoolExecutor(max_workers=50) as executor:
    all_ingr = {ingr:nutr for ingr, nutr in zip(list_of_food, list(tqdm(executor.map(getInfo, list_of_food), total=len(list_of_food))))}

  0%|          | 0/176 [00:00<?, ?it/s]

In [15]:
df_list_of_nutr = pd.DataFrame(all_ingr).T
df_list_of_nutr.fillna(0, inplace=True)

In [16]:
df_list_of_nutr

,Protein,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,...,Thiamin,Riboflavin,Niacin,Folate,Choline,Vitamin E,Cholesterol,Pantothenic acid,Manganese,Biotin
almond,41.34,20.230769,22.666667,63.333333,40.32,15.765957,10.086957,28.727273,107.666667,1.454545,...,7.833333,71.769231,24.28750,10.50,9.418182,0.0,0.000000,0.00,0.0,0.0
amaretto,0.00,0.000000,0.000000,0.000000,0.00,0.000000,2.913043,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.0
anchovy,57.78,17.846154,25.722222,16.428571,20.16,11.574468,159.478261,22.181818,37.666667,123.818182,...,6.500000,27.923077,124.39375,3.25,15.454545,0.0,28.333333,0.00,0.0,0.0
anise,35.20,49.692308,205.555556,40.476190,35.20,30.638298,0.695652,48.181818,101.111111,9.090909,...,28.333333,22.307692,19.12500,2.50,0.000000,0.0,0.000000,15.94,100.0,0.0
apple,0.00,0.000000,1.277778,0.000000,0.00,2.340426,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
watermelon,1.56,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.0
wild rice,26.60,0.000000,8.888889,0.000000,26.64,5.680851,0.000000,30.272727,0.000000,0.000000,...,0.000000,17.461538,41.68750,0.00,0.000000,0.0,0.000000,0.00,0.0,0.0
yellow squash,2.00,0.769231,2.000000,0.000000,0.00,0.000000,0.434783,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.0
yogurt,7.04,10.153846,0.000000,3.095238,0.00,3.276596,2.086957,5.636364,0.000000,0.000000,...,0.000000,11.538462,0.00000,0.00,0.000000,0.0,5.000000,0.00,0.0,0.0


In [17]:
df_list_of_nutr.to_csv("../datasets/nutrients.csv")

In [18]:
df_rec.dropna(inplace=True)

In [19]:
def fetch_link(title):
    title = title.strip()
    try:
        with requests.Session() as session:
            response = session.get(
                "https://www.epicurious.com/search", 
                params={"q": title},
                timeout=10,
                headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
            )
            soup = BeautifulSoup(response.text, 'lxml')
            lnk = soup.find(class_="SummaryCollectionGridItems-DZShR fVPNxz hide-read-more-ad")
            for card in lnk:
                if card['class'][0:4] == ['StackedRatingsCardWrapper-fRZEyp', 'kxAQph', 'SummaryCollectionGridSummaryItem-WColm', 'etOKhn']:
                    link = card.find(class_ = "BaseWrap-sc-gjQpdd BaseText-ewhhUZ BaseLink-eNWuiM Link-ehXskl iUEiRd bjMGUM kxxKIa")
                    if link.find(class_ = "BaseWrap-sc-gjQpdd BaseText-ewhhUZ Hed-hfYgrM iUEiRd euMCMz hqUziD").text == title:
                        return f"https://www.epicurious.com{link['href']}"
            return None
    except Exception:
        return None




In [21]:
def fast_parse(df):
    with ThreadPoolExecutor(max_workers=50) as executor: 
        return list(tqdm(executor.map(fetch_link, df['title']), total=len(df)))

links = fast_parse(df_rec)

  0%|          | 0/15864 [00:00<?, ?it/s]

In [22]:
df_rec['link'] = links

In [83]:
df_rec

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey,link
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...


In [ ]:
for i in ['protein', 'fat', 'sodium']:
    df_rec[i] = df_rec[i]/nutrients_list[i.capitalize()]*100

In [88]:
df_rec

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey,link
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,60.0,8.974359,24.304348,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,36.0,29.487179,62.565217,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
2,Potato and Fennel Soup Hodge,3.750,165.0,12.0,8.974359,7.173913,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
4,Spinach Noodle Casserole,3.125,547.0,40.0,41.025641,19.652174,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
5,The Best Blts,4.375,948.0,38.0,101.282051,45.304348,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,4.0,2.564103,2.782609,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
20048,Artichoke and Parmesan Risotto,4.375,671.0,44.0,35.897436,25.347826,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
20049,Turkey Cream Puff Pie,4.375,563.0,62.0,48.717949,28.347826,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,90.0,30.769231,22.478261,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...


In [89]:
final_df = pd.concat([df_rec.drop(axis=1, labels=list(df_rec.iloc[0, 6:].index)), df_rec[['breakfast', 'lunch', 'dinner', 'link']], df_rec[list_of_food]], axis=1)

In [90]:
final_df

,title,rating,calories,protein,fat,sodium,breakfast,lunch,dinner,link,...,vanilla,veal,vegetable,walnut,wasabi,watermelon,wild rice,yellow squash,yogurt,zucchini
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,60.0,8.974359,24.304348,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,36.0,29.487179,62.565217,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,12.0,8.974359,7.173913,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,40.0,41.025641,19.652174,0.0,0.0,0.0,None,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,The Best Blts,4.375,948.0,38.0,101.282051,45.304348,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,4.0,2.564103,2.782609,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,44.0,35.897436,25.347826,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,62.0,48.717949,28.347826,0.0,0.0,1.0,None,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,90.0,30.769231,22.478261,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
final_df[final_df.iloc[:, 10:].apply(lambda x: x.sum(), axis=1) > 0].dropna().drop_duplicates(subset='title').to_csv("../datasets/epi_r_clean_links.csv", index=False)

In [92]:
itog = pd.read_csv("../datasets/epi_r_clean_links.csv")

In [93]:
itog

,title,rating,calories,protein,fat,sodium,breakfast,lunch,dinner,link,...,vanilla,veal,vegetable,walnut,wasabi,watermelon,wild rice,yellow squash,yogurt,zucchini
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,60.0,8.974359,24.304348,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,36.0,29.487179,62.565217,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,12.0,8.974359,7.173913,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,The Best Blts,4.375,948.0,38.0,101.282051,45.304348,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ham Persillade with Mustard Potato Salad and M...,3.750,602.0,46.0,52.564103,73.739130,0.0,1.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,Orange-Tini,3.750,84.0,0.0,0.000000,0.217391,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10328,Chinese Barbecued Spareribs,3.750,998.0,110.0,102.564103,88.130435,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10329,Artichoke and Parmesan Risotto,4.375,671.0,44.0,35.897436,25.347826,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10330,Snapper on Angel Hair with Citrus Cream,4.375,631.0,90.0,30.769231,22.478261,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
